In [1]:
import json
import pathlib
from IPython.display import display, Markdown

import toml
import pandas as pd
import mlflow
import mlflow.sklearn
from hyperopt.pyll import scope
from tpot import TPOTClassifier
from imblearn.combine import SMOTETomek
from hyperopt import (fmin, tpe,
                      hp, Trials,
                      STATUS_OK)

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (precision_score, recall_score, f1_score,
                             classification_report, confusion_matrix,
                             average_precision_score)

# Read TPOT configuration from toml file
config = toml.load("../config.toml")
paths = config['paths']
mlflow_tracking_uri = pathlib.Path(paths['mlflow']['tracking_uri'])
SEED = config["settings"]["general"]["seed"]

# Load the dataset
df = pd.read_parquet(paths['pre']["data"])

# Display the first few rows to ensure the data is loaded correctly
display(Markdown(f'The dataset has {df.shape[0]:,} rows and {df.shape[1]:,} columns.'))
display(Markdown(f'The dataset has the following columns: {", ".join(df.columns)}.'))
display(df.sample(3))
display(df.describe())

The dataset has 40,000 rows and 22 columns.

The dataset has the following columns: y, job_blue-collar, job_entrepreneur, job_housemaid, job_management, job_retired, job_self-employed, job_services, job_student, job_technician, job_unemployed, marital_married, marital_single, education_secondary, education_tertiary, default_yes, housing_yes, loan_yes, contact_telephone, high_balance, age_group_18-30, age_group_60+.

,y,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,...,marital_single,education_secondary,education_tertiary,default_yes,housing_yes,loan_yes,contact_telephone,high_balance,age_group_18-30,age_group_60+
37202,0,1,0,0,0,0,0,0,0,0,...,1,1,0,0,1,0,0,0,1,0
9033,0,0,0,0,0,0,1,0,0,0,...,1,1,0,0,0,0,0,0,0,0
6079,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,1,0


,y,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,...,marital_single,education_secondary,education_tertiary,default_yes,housing_yes,loan_yes,contact_telephone,high_balance,age_group_18-30,age_group_60+
count,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.00000,40000.000000,40000.000000,...,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000
mean,0.072400,0.234575,0.035125,0.027175,0.204150,0.035925,0.035350,0.09775,0.013100,0.171300,...,0.272225,0.524825,0.280150,0.020225,0.600775,0.173250,0.058025,0.397950,0.145975,0.005850
std,0.259152,0.423738,0.184098,0.162595,0.403084,0.186106,0.184665,0.29698,0.113704,0.376776,...,0.445111,0.499390,0.449078,0.140771,0.489745,0.378468,0.233794,0.489481,0.353086,0.076262
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [2]:
# Separate features and target variable
y = (pd.read_parquet(paths['post']["data"], columns=['number_calls'])==1).astype(int)
# y = df['y']  # Target variable
X = df.drop('y', axis=1)  # Features

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Identify numerical and categorical columns
numerical_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = X_train.select_dtypes(include=['uint8']).columns.tolist()  # Treat uint8 as categorical

# Debugging: Check if the columns are being correctly identified
print(f"Categorical columns (uint8): {categorical_cols}")
print(f"Numerical columns: {numerical_cols}")

# Preprocessing pipeline for numerical data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Preprocessing pipeline for categorical data (if needed, though it may not require any transformation)
# We assume categorical columns are already encoded, so no OneHotEncoder is used here.
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))  # Just in case there are any NaNs
])

# Combine preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Apply preprocessing to both numerical and categorical columns
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# Convert the processed data into a DataFrame with correct column names
X_train_processed = pd.DataFrame(X_train_processed, columns=preprocessor.get_feature_names_out())
X_test_processed = pd.DataFrame(X_test_processed, columns=preprocessor.get_feature_names_out())

# Apply SMOTETomek on the training data
smote_tomek = SMOTETomek(random_state=42)
X_train_balanced, y_train_balanced = smote_tomek.fit_resample(X_train_processed, y_train)

# Convert back to DataFrame if needed
df_train_balanced = pd.DataFrame(X_train_balanced, columns=preprocessor.get_feature_names_out())

# Now your balanced data is ready for further use
df_train_balanced

Categorical columns (uint8): ['job_blue-collar', 'job_entrepreneur', 'job_housemaid', 'job_management', 'job_retired', 'job_self-employed', 'job_services', 'job_student', 'job_technician', 'job_unemployed', 'marital_married', 'marital_single', 'education_secondary', 'education_tertiary', 'default_yes', 'housing_yes', 'loan_yes', 'contact_telephone', 'age_group_18-30', 'age_group_60+']
Numerical columns: []


,cat__job_blue-collar,cat__job_entrepreneur,cat__job_housemaid,cat__job_management,cat__job_retired,cat__job_self-employed,cat__job_services,cat__job_student,cat__job_technician,cat__job_unemployed,cat__marital_married,cat__marital_single,cat__education_secondary,cat__education_tertiary,cat__default_yes,cat__housing_yes,cat__loan_yes,cat__contact_telephone,cat__age_group_18-30,cat__age_group_60+
0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0
2,0,0,0,1,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57249,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0
57250,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0
57251,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0
57252,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,1,0,0


In [3]:
# Define a function to log parameters, metrics, and model to MLflow
def log_results(y_test, y_pred, model, params):
    # Log hyperparameters
    mlflow.log_param('n_estimators', params.get('n_estimators', 'N/A'))  # Handle missing params for TPOT
    mlflow.log_param('max_depth', params.get('max_depth', 'N/A'))  # Handle missing params for TPOT
    mlflow.log_param('min_samples_split', params.get('min_samples_split', 'N/A'))  # Handle missing params for TPOT

    # Log key metrics
    mlflow.log_metric('precision', precision_score(y_test, y_pred))
    mlflow.log_metric('recall', recall_score(y_test, y_pred))
    mlflow.log_metric('f1_score', f1_score(y_test, y_pred))
    mlflow.log_metric('average_precision', average_precision_score(y_test, y_pred))

    # Log classification report as a text artifact
    mlflow.log_text(classification_report(y_test, y_pred), artifact_file="classification_report.txt")

    # Log confusion matrix as a text artifact
    mlflow.log_text(str(confusion_matrix(y_test, y_pred)), artifact_file="confusion_matrix.txt")

    # Log the model
    mlflow.sklearn.log_model(model, "model")

# Define the objective function
def objective(params):
    with mlflow.start_run(nested=True):  # Create a new nested run for each evaluation
        # Extract hyperparameters
        n_estimators = int(params['n_estimators'])
        max_depth = int(params['max_depth'])
        min_samples_split = int(params['min_samples_split'])

        # Define and train the model
        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            random_state=42
        )
        model.fit(X_train, y_train)

        # Make predictions and calculate metrics
        y_pred = model.predict(X_test)
        precision = precision_score(y_test, y_pred)

        # Log results to MLflow
        log_results(y_test, y_pred, model, params)

        # Return a dictionary with status and loss (to minimize)
        return {'loss': -precision, 'status': STATUS_OK}

# Define the search space for Hyperopt
space = {
    'n_estimators': scope.int(hp.quniform('n_estimators', 50, 200, 1)),
    'max_depth': scope.int(hp.quniform('max_depth', 5, 50, 1)),
    'min_samples_split': scope.int(hp.quniform('min_samples_split', 2, 10, 1)),
}

# Create a Trials object to store information about the optimization process 
trials = Trials()

# Start the MLflow run and capture the existing run ID and tracking URI
with mlflow.start_run() as run:
    run_id = run.info.run_id
    mlflow.set_tracking_uri(mlflow_tracking_uri)

    # Run Hyperopt
    best = fmin(
        fn=objective,  # Objective function
        space=space,  # Search space
        algo=tpe.suggest,  # Tree-structured Parzen Estimator
        max_evals=50,  # Number of evaluations (can be adjusted)
        trials=trials  # Store results of each evaluation
    )

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

c:\Users\Guill\miniconda3\envs\term_deposit\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)

2024/10/02 09:35:32 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



  2%|▏         | 1/50 [00:10<08:20, 10.21s/trial, best loss: -0.8942548921224285]

c:\Users\Guill\miniconda3\envs\term_deposit\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)

2024/10/02 09:35:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



  4%|▍         | 2/50 [00:16<06:15,  7.83s/trial, best loss: -0.8942548921224285]

...

 96%|█████████▌| 48/50 [04:31<00:11,  5.59s/trial, best loss: -0.8943732730469731]

c:\Users\Guill\miniconda3\envs\term_deposit\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)

2024/10/02 09:39:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



 98%|█████████▊| 49/50 [04:37<00:05,  5.68s/trial, best loss: -0.8943732730469731]

c:\Users\Guill\miniconda3\envs\term_deposit\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)

2024/10/02 09:40:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



100%|██████████| 50/50 [04:43<00:00,  5.67s/trial, best loss: -0.8943732730469731]


In [86]:
print(f"Best Hyperparameters: {best}")
# print(f"Existing Run ID: {run_id}")
print(f"MLflow Tracking URI: {mlflow_tracking_uri}")

# Calculate precision, recall, and F1 score
model = RandomForestClassifier(
    n_estimators=int(best['n_estimators']),
    max_depth=int(best['max_depth']),
    min_samples_split=int(best['min_samples_split']),
    random_state=42
)
model.fit(df_train_balanced, y_train_balanced)
y_pred = model.predict(X_test)

Best Hyperparameters: {'max_depth': 27.0, 'min_samples_split': 2.0, 'n_estimators': 81.0}
MLflow Tracking URI: ../mlruns/pre/


c:\Users\Guill\miniconda3\envs\term_deposit\lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [101]:
# Calculate the True possitive rate
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

# Display the confusion matrix
display(Markdown("Confusion Matrix _(normalized: True):_"))
display(pd.DataFrame(
    data=confusion_matrix(y_test, y_pred, normalize='true').round(3),
    columns=["0", "1"],
    index=["0", "1"]
    )
)

print(f"F1 Score: {f1_score(y_test, y_pred):.3f}") # Calculate the F1 score
print(f"precision_score: {precision_score(y_test, y_pred):.3f}")  # Calculate the precision
print(f"True Positive Rate: {tp / (tp + fn):.3f}")

# Calculate the False possitive rate
print(f"False Positive Rate: {fp / (fp + tn):.3f}")
display(Markdown("Confusion Matrix _(normalized: pred):_"))
cm_pred = confusion_matrix(y_test, y_pred, normalize='pred')
display(pd.DataFrame(cm_pred, columns=["0", "1"], index=["0", "1"]))

Confusion Matrix _(normalized: True):_

,0,1
0,0.475,0.525
1,0.397,0.603


F1 Score: 0.725
precision_score: 0.907
True Positive Rate: 0.603
False Positive Rate: 0.525


Confusion Matrix _(normalized: pred):_

,0,1
0,0.124112,0.093468
1,0.875888,0.906532


## TPOT

Exploring different models with TPOT:

1. Config 1 as a 5 generations default TPOT config scoring over f1.

In [22]:
def ensure_path(path, filename):
    path = Path(path).joinpath(filename)
    path.parent.mkdir(parents=True, exist_ok=True)
    return path

path_tpot = Path(paths["TPOT"]["base"])

# Define the TPOT configuration
path_checkpoint = ensure_path(path=paths["TPOT"]["periodic_checkpoint_folder"],
            filename="checkpoint_{run_id}")

path_pipeline = ensure_path(path_tpot, filename="pipelines/pipeline_{run_id}.py")
path_log = ensure_path(path_tpot, filename="logs/log_{run_id}.txt")

In [ ]:
tpot = TPOTClassifier(**config["TPOT"]["one"],
                      log_file = path_log,
                      periodic_checkpoint_folder = path_checkpoint,
                      )

# Ensure target labels are in 1-D array format
if isinstance(y_train_balanced, pd.DataFrame):
    y_train_balanced = y_train_balanced["number_calls"]  # Convert to pandas Series (1-D array)

# Now fit TPOT with the balanced training data
tpot.fit(df_train_balanced, y_train_balanced)

In [7]:
# Score the model
print("Test Score:", tpot.score(X_test_processed, y_test))

# Export the generated pipeline
tpot.export(path_pipeline)

# Log the TPOT pipeline
with open(path_log, 'r') as file:
    pipeline_code = file.read()  # Read the pipeline code
    mlflow.log_text(pipeline_code, artifact_file="pipeline_code.py")

c:\Users\Guill\miniconda3\envs\term_deposit\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Test Score: 0.9076280041797283


In [8]:
# Set a new experiment for TPOT
tpot_experiment_name = "TPOT_Model_Experiment"
mlflow.set_experiment(tpot_experiment_name)

# Assume y_pred is obtained from the trained TPOT model
y_pred = tpot.predict(X_test_processed)

# Set the tracking URI if needed (optional)
mlflow.set_tracking_uri(mlflow_tracking_uri)  # Use the captured tracking URI

# Log metrics to a new run in the TPOT experiment
with mlflow.start_run(nested=True) as run:  # Start a new run for the TPOT experiment
    # Log results using the log_results function
    log_results(y_test, y_pred, tpot.fitted_pipeline_, {})  # Pass empty params for TPOT since they may not exist

print("TPOT metrics and artifacts have been logged to a new MLflow experiment.")

2024/10/02 11:01:22 INFO mlflow.tracking.fluent: Experiment with name 'TPOT_Model_Experiment' does not exist. Creating a new experiment.


2024/10/02 11:01:43 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


TPOT metrics and artifacts have been logged to a new MLflow experiment.


## Experiment 2

In [10]:
with open(path_tpot.join("classifier_config_dict.json"), "r") as file:
    classifier_config_dict = json.load(file)
    
# change the types to int8 for X_train
X_train = X_train.astype('int8')
y_train = y_train.astype('int8')

# balance the datasets using SMOTEENN
smote_tomek = SMOTETomek(random_state=42)

X_train_balanced, y_train_balanced = smote_tomek.fit_resample(X_train, y_train)

# convert the balanced datasets to DataFrame
df_train_balanced = pd.DataFrame(X_train_balanced, columns=X_train.columns)

In [19]:
# create a new experiment for the classifier
classifier_experiment_name = "Classifier_Model_Experiment"

tpot = TPOTClassifier(**config["TPOT"]["two"],
                      config_dict=classifier_config_dict)
tpot.fit(df_train_balanced, y_train_balanced)

c:\Users\Guill\miniconda3\envs\term_deposit\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Optimization Progress:   0%|          | 0/100 [00:00<?, ?pipeline/s]


Best pipeline: KNeighborsClassifier(ZeroCount(input_matrix), n_neighbors=93, p=2, weights=distance)


TPOTClassifier(config_dict={'sklearn.cluster.FeatureAgglomeration': {'affinity': ['euclidean',
                                                                                  'l1',
                                                                                  'l2',
                                                                                  'manhattan',
                                                                                  'cosine'],
                                                                     'linkage': ['ward',
                                                                                 'complete',
                                                                                 'average']},
                            'sklearn.decomposition.FastICA': {'tol': [0.0, 0.05,
                                                                      0.1,
                                                                      0.15000000000000002,
                                                                      0.2, 0.25,
                                                                      0.30000000000000004,
                                                                      0.35000000000000003,
                                                                      0.4, 0.45,
                                                                      0.5, 0.55,
                                                                      0.6000000000000001,
                                                                      0.65,
                                                                      0.7000000000000001,
                                                                      0....
                                                                                          4,
                                                                                          5,
                                                                                          6,
                                                                                          7,
                                                                                          8,
                                                                                          9,
                                                                                          10,
                                                                                          11,
                                                                                          12,
                                                                                          13,
                                                                                          14,
                                                                                          15,
                                                                                          16,
                                                                                          17,
                                                                                          18,
                                                                                          19,
                                                                                          20]},
                            'tpot.builtins.OneHotEncoder': {'minimum_fraction': [0.05,
                                                                                 0.1,
                                                                                 0.15,
                                                                                 0.2,
                                                                                 0.25],
                                                            'sparse': [False],
                                                            'threshold': [10]}, ...},
               generations=1, log_file='../models/tpot/pre/term_depo

In [56]:
# Set a new experiment for TPOT
tpot_experiment_name = "TPOT_Model_Experiment"
mlflow.set_experiment(tpot_experiment_name)

# Assume y_pred is obtained from the trained TPOT model
y_pred = tpot.predict(X_test)

# Set the tracking URI if needed (optional)
# mlflow.set_tracking_uri(mlflow_tracking_uri)  # Use the captured tracking URI

# Log metrics to a new run in the TPOT experiment
with mlflow.start_run(nested=True) as run:  # Start a new run for the TPOT experiment
    # Log results using the log_results function
    log_results(y_test, y_pred, tpot.fitted_pipeline_, {})  # Pass empty params for TPOT since they may not exist

print("TPOT metrics and artifacts have been logged to a new MLflow experiment.")

# Score the model
print("Test Score:", tpot.score(X_test, y_test))
print("Report:\n", classification_report(y_test, y_pred))  # report

# Export the generated pipeline
tpot.export(path_tpot.joinpath("precall_pipeline_2.py"))

c:\Users\Guill\miniconda3\envs\term_deposit\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Test Score: 0.9048305695746215
Report:
               precision    recall  f1-score   support

           0       0.12      0.53      0.19       847
           1       0.90      0.53      0.67      7153

    accuracy                           0.53      8000
   macro avg       0.51      0.53      0.43      8000
weighted avg       0.82      0.53      0.62      8000



In [103]:
# Calculate the True possitive rate
y_pred = tpot.predict(X_test)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

# Display the confusion matrix
display(Markdown("Confusion Matrix _(normalized: True):_"))
display(pd.DataFrame(
    data=confusion_matrix(y_test, y_pred, normalize='true').round(3),
    columns=["0", "1"],
    index=["0", "1"]
    )
)

print(f"F1 Score: {f1_score(y_test, y_pred):.3f}")  # Calculate the F1 score
print(f"precision_score: {precision_score(y_test, y_pred):.3f}")  # Calculate the precision
print(f"True Positive Rate: {tp / (tp + fn):.3f}")

# Calculate the False possitive rate
print(f"False Positive Rate: {fp / (fp + tn):.3f}")
display(Markdown("Confusion Matrix _(normalized: pred):_"))
cm_pred = confusion_matrix(y_test, y_pred, normalize='pred')
display(pd.DataFrame(cm_pred, columns=["0", "1"], index=["0", "1"]))

Confusion Matrix _(normalized: True):_

,0,1
0,0.532,0.468
1,0.474,0.526


F1 Score: 0.666
precision_score: 0.905
True Positive Rate: 0.526
False Positive Rate: 0.468


Confusion Matrix _(normalized: pred):_

,0,1
0,0.117479,0.095169
1,0.882521,0.904831


## False negative rate: The Missed 1's

**Understanding the Problem:**
We want to determine the percentage of actual class 1 instances that the model failed to predict. This is essentially the **false negative rate**.

# Analizing results

### Key Comparisons:

1. **F1 Score**:
   - Hyperopt: 0.725
   - TPOT: 0.666
   - **Winner**: Hyperopt. The F1 score is a key measure of overall balance between precision and recall, and Hyperopt performs better here.
2. **Precision Score**:
   - Hyperopt: 0.907
   - TPOT: 0.905
   - **Winner**: Hyperopt (slightly). Precision measures how many of the predicted positives are correct. Hyperopt performs slightly better.

In [66]:
# def convert_to_list(d):
#     if isinstance(d, dict):
#         return {k: convert_to_list(v) for k, v in d.items()}
#     elif isinstance(d, list):
#         return [convert_to_list(i) for i in d]
#     elif isinstance(d, (range, np.ndarray)):
#         return list(d)
#     else:
#         return d

# with open("../models/tpot/pre/classifier_config_dict.json", "w") as json_file:
#     json.dump(convert_to_list(classifier_config_dict), json_file, indent=4)

# next steps

Predict on the entire dataset to calculate the profit increase:
- Human hours saved
- Calls reduction
- Deposit increase increase in a week
- Percentage of calls lost

Customer segmentation
- Try different techniques
- TSNE, PCA, uMAP